# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [0]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [0]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [0]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#  total number of rows 
print("the total number of rows is {}".format(len(full_data_rows_list)))


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


the total number of rows is 8056


In [0]:
# check the number of rows in the  csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId



## The Apache Cassandra code starts below

#### Creating a Cluster

In [0]:
# Making  a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [0]:
#Creating a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [0]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')


## queries were created below to answer the following three questions 

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [0]:
'''the best approach to cassandra and SQL queries is to create functions that perform just that to avoid excessive number of code lines'''

def execute_create_drop_query(query):
  ''' 
  This function will act as a short cut to execute CREATE or DROP tables
  It takes cassandra CQL queries as input and outputs the execution of the queries
  '''
    try:
        session.execute(query)
    except Exception as e:
        print(e)
        
        
def execute_select_query(query):
  ''' 
  This function will act as a short cut to execute CREATE or DROP tables
  It takes cassandra CQL queries as input and outputs the execution of the queries
  '''
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

# Convert tuple into a list to see values
    for row in rows:
        
        print (list(row))

***
### Note:
The table we need to create to answer the first question will use artist_name, song_name, song_length. 

I am not sure if I can use random names that don't match with what's inside the database columns. Hmm. To be safe, I will use song, length, and the rest of the column names as described in the csv file.

In addition, sessionId, itemInSession will be used as partition and primary keys.

The reason we use them together because sessionId alone is not unique, you need iteminsession to make it unique and easy to use with queries.
***

In [0]:
## The code below Answers Question 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

execute_create_drop_query("""
    CREATE TABLE IF NOT EXISTS song_sessions (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
""")    

In [0]:
# The code below will do the insertion tasks
file = 'event_datafile_new.csv'

# Inserting data into song_plays_by_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_sessions (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [0]:
## a SELECT statement to verify the data was entered into the table
execute_select_query("""
    SELECT artist, song, length
    FROM song_sessions
    WHERE sessionId=338 AND itemInSession=4
""")

['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


as we can see, we have retrieved the data as expected; Our query answers question 1. 

#### This is the end of question 1 task

***
##  Question 2:
question 2 asks the following:

2. Give me the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


to answer this question, similar to the work we have done above, we need to create user_session table with the following columns:

        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        
the primary key should be the userId and the clustering should be done using the session id and iteminsession
***

In [0]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
execute_create_drop_query("""
    CREATE TABLE IF NOT EXISTS user_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY (userId, sessionId, itemInSession)
    )
""")

                    

In [0]:
#inserting data
file = 'event_datafile_new.csv'

# Insert expected data into song_plays_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [0]:
#executing select query 
execute_select_query("""
    SELECT itemInSession, artist, song, firstName, lastName
    FROM user_session 
    WHERE userId=10 AND sessionId=182
""")

[0, 'Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz']
[1, 'Three Drives', 'Greece 2000', 'Sylvie', 'Cruz']
[2, 'Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz']
[3, 'Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz']


#### Note:
as we can see, we have retrieved the data as expected; Our query answers question 2. our method works! let's move to Q3
***


####  question 3 asks the following:

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




to answer this question, similar to the work we have done above, we need to create user_session table with the following columns:

        song text,
        userId int,
        firstName text,
        lastName text,
        
the primary key should be the song and userId
***

In [0]:
## The code below answers Query 3: 'Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own''
execute_create_drop_query("""
    CREATE TABLE IF NOT EXISTS username_retrieve (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY (song, userId)
    )
""")

file = 'event_datafile_new.csv'

# This code inserts expected data into users_by_song_listens
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_retrieve (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))                    

In [0]:
#This code performs the select query
execute_select_query("""
    SELECT userId, firstName, lastName
    FROM username_retrieve
    WHERE song='All Hands Against His Own'
""")

[29, 'Jacqueline', 'Lynch']
[80, 'Tegan', 'Levine']
[95, 'Sara', 'Johnson']


***

As we can see above, the select queries have retrieved the required information. We are done!

### Drop the tables before closing out the sessions

In [0]:
#The code below drops the tables that we created

execute_create_drop_query("DROP TABLE IF EXISTS song_sessions")

execute_create_drop_query("DROP TABLE IF EXISTS user_session")



execute_create_drop_query("DROP TABLE IF EXISTS username_retrieve")


### Close the session and cluster connection¶

In [0]:
#the code below shuts down the session and the cluster we created
session.shutdown()
cluster.shutdown()